# Visualize My Caches
Application uses Folium to map caches

In [38]:
# Import standard libraries
import re
import xml.etree.ElementTree as ET

# Import external libraries
import folium
import numpy as np
import pandas as pd

## Import the Geocaching data file
You can generate and download a pocket query containing all your geocaching logs.

In [4]:
pQueryFile = '391079.gpx'
tree = ET.parse(pQueryFile)
root = tree.getroot()

## XML Format
The pocket query has this format:
```xml
<gpx xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="1.0" creator="Groundspeak Pocket Query" xsi:schemaLocation="http://www.topografix.com/GPX/1/0 http://www.topografix.com/GPX/1/0/gpx.xsd http://www.groundspeak.com/cache/1/0/1 http://www.groundspeak.com/cache/1/0/1/cache.xsd" xmlns="http://www.topografix.com/GPX/1/0">
  <name>My Finds Pocket Query</name>
  <desc>Geocache file generated by Groundspeak</desc>
  <author>Groundspeak</author>
  <email>contact@groundspeak.com</email>
  <time>2018-11-14T00:16:12.1350479Z</time>
  <keywords>cache, geocache, groundspeak</keywords>
  <bounds minlat="26.063217" minlon="-118.640633" maxlat="44.570867" maxlon="-70.8668" />
  <wpt lat="33.745217" lon="-112.090783">
      ... See below ....
  </wpt>
</gpx>
```

There will be multiple `<wpt>` entries, each one with the following format:
```xml
  <wpt lat="33.745217" lon="-112.090783">
    <time>2010-05-26T07:00:00Z</time>
    <name>GC2988Z</name>
    <desc>Sonoran Trail # 3, "Lets sit here" by kira's rangers, Traditional Cache (2/3.5)</desc>
    <url>http://www.geocaching.com/seek/cache_details.aspx?guid=237c350f-8f30-4885-82ab-923fcf6d4a91</url>
    <urlname>Sonoran Trail # 3, "Lets sit here"</urlname>
    <sym>Geocache Found</sym>
    <type>Geocache|Traditional Cache</type>
    <groundspeak:cache id="1712007" available="False" archived="True" xmlns:groundspeak="http://www.groundspeak.com/cache/1/0/1">
      <groundspeak:name>Sonoran Trail # 3, "Lets sit here"</groundspeak:name>
      <groundspeak:placed_by>kira's rangers</groundspeak:placed_by>
      <groundspeak:owner id="2037845">kira's rangers</groundspeak:owner>
      <groundspeak:type>Traditional Cache</groundspeak:type>
      <groundspeak:container>Micro</groundspeak:container>
      <groundspeak:attributes>
        <groundspeak:attribute id="34" inc="0">Quads</groundspeak:attribute>
        <groundspeak:attribute id="33" inc="0">Motorcycles</groundspeak:attribute>
        <groundspeak:attribute id="35" inc="0">Off-road vehicles</groundspeak:attribute>
        <groundspeak:attribute id="32" inc="1">Bicycles</groundspeak:attribute>
        <groundspeak:attribute id="1" inc="1">Dogs</groundspeak:attribute>
      </groundspeak:attributes>
      <groundspeak:difficulty>2</groundspeak:difficulty>
      <groundspeak:terrain>3.5</groundspeak:terrain>
      <groundspeak:country>United States</groundspeak:country>
      <groundspeak:state>Arizona</groundspeak:state>
      <groundspeak:short_description html="False">short cache description</groundspeak:short_description>
      <groundspeak:long_description html="False">L-O-N-G cache description</groundspeak:long_description>
      <groundspeak:encoded_hints>
      </groundspeak:encoded_hints>
      <groundspeak:logs>
        <groundspeak:log id="459354118">
          <groundspeak:date>2014-11-04T20:00:00Z</groundspeak:date>
          <groundspeak:type>Found it</groundspeak:type>
          <groundspeak:finder id="30844">wsp2</groundspeak:finder>
          <groundspeak:text encoded="False">Your log entry</groundspeak:text>
        </groundspeak:log>
      </groundspeak:logs>
      <groundspeak:travelbugs />
    </groundspeak:cache>
  </wpt>
```



In [51]:
# Add the namespaces to make life easier
ns = {'gpx': 'http://www.topografix.com/GPX/1/0',
      'gc':  'http://www.groundspeak.com/cache/1/0/1',}

In [14]:
# Extract some of the header info
pqName = root.find('gpx:name', ns).text
pqDesc = root.find('gpx:desc', ns).text
pqTime = root.find('gpx:time', ns).text
pqBounds = root.find('gpx:bounds', ns).attrib

# Find the center of the map using the bounds of the query
mapCenter = [np.mean([float(pqBounds['minlat']), float(pqBounds['maxlat'])]), 
             np.mean([float(pqBounds['minlon']), float(pqBounds['maxlon'])])]

## Create dataframe with pocket query waypoint data

In [68]:
#columns = ['GCName', 'Lat', 'Lon', 'Time', 'Type', 'Name', 'Container', 'Difficulty', 'Terrain', 'Country', 'State', 'LogType', 'LogDate']
#df = pd.DataFrame(columns=columns)

logList = []

In [70]:
entries = root.findall('gpx:wpt', ns)
for cache in root.findall('gpx:wpt', ns):
    for log in cache.findall('gc:cache/gc:logs/gc:log', ns):
        d = {}
        d['GCName']     = cache.find('gpx:name', ns).text
        d['Lat']        = float(cache.attrib['lat'])
        d['Lon']        = float(cache.attrib['lon'])
        d['Time']       = pd.Timestamp(cache.find('gpx:time', ns).text)
        d['Name']       = cache.find('gc:cache/gc:name', ns).text
        d['Container']  = cache.find('gc:cache/gc:container', ns).text
        d['Difficulty'] = float(cache.find('gc:cache/gc:difficulty', ns).text)
        d['Terrain']    = float(cache.find('gc:cache/gc:terrain', ns).text)
        d['Country']    = cache.find('gc:cache/gc:country', ns).text
        d['State']      = cache.find('gc:cache/gc:state', ns).text
        d['LogType']    = log.find('gc:type', ns).text
        d['LogDate']    = pd.Timestamp(log.find('gc:date', ns).text)
    logList.append(d)
    

In [71]:
df = pd.DataFrame(logList)

In [73]:
df

,Container,Country,Difficulty,GCName,Lat,LogDate,LogType,Lon,Name,State,Terrain,Time
0,Micro,United States,2.0,GC2988Z,33.745217,2014-11-04 20:00:00+00:00,Found it,-112.090783,"Sonoran Trail # 3, ""Lets sit here""",Arizona,3.5,2010-05-26 07:00:00+00:00
1,Micro,United States,2.0,GC6E670,33.702450,2018-10-25 19:00:00+00:00,Found it,-111.955917,"AZ Stickman ""five""",Arizona,2.0,2016-03-15 07:00:00+00:00
2,Micro,United States,2.0,GC6EAKY,33.714333,2018-10-25 19:00:00+00:00,Found it,-111.958683,"AZ Stickman ""twentyfour""",Arizona,2.0,2016-03-15 07:00:00+00:00
3,Micro,United States,2.0,GC6EAVN,33.716933,2018-10-25 19:00:00+00:00,Found it,-111.950100,"AZ Stickman ""thirtyfour""",Arizona,2.0,2016-03-15 07:00:00+00:00
4,Micro,United States,2.0,GC6EDEF,33.718833,2018-10-25 19:00:00+00:00,Found it,-111.959833,"AZ Stickman ""sixty""",Arizona,2.0,2016-03-15 07:00:00+00:00
5,Micro,United States,1.5,GC3G9D4,35.364433,2014-07-20 19:00:00+00:00,Found it,-94.377050,"ROCKS, ROCKS, ROCKS",Arkansas,2.0,2012-04-05 07:00:00+00:00
6,Small,United States,1.5,GC3R730,35.396233,2014-07-20 19:00:00+00:00,Found it,-94.368833,The Ent,Arkansas,1.5,2012-07-24 07:00:00+00:00
7,Micro,United States,2.0,GC3R73H,35.395017,2014-07-20 19:00:00+00:00,Found it,-94.371217,Weeping,Arkansas,1.5,2012-07-24 07:00:00+00:00
8,Other,United States,1.5,GC28YKP,34.004433,2012-07-13 18:13:52+00:00,Found it,-118.486950,Community (Garden) Cache,California,1.0,2010-06-02 07:00:00+00:00
9,Regular,United States,2.0,GC2MRFA,38.947433,2014-06-17 19:00:00+00:00,Found it,-105.381400,Animal Den,Colorado,2.5,2011-01-22 08:00:00+00:00


## Begin building the map

In [31]:
cacheMap = folium.Map(location=mapCenter, zoom_start=2)
# fit_bounds takes list of two coords: southwest and northeast
cacheMap.fit_bounds([[float(pqBounds['minlat']), float(pqBounds['minlon'])], 
                     [float(pqBounds['maxlat']), float(pqBounds['maxlon'])]])

#cacheMap

## Add geocaches to the map

In [37]:
# instantiate a feature group for the geocaches
caches = folium.map.FeatureGroup()

for cache in root.findall('gpx:wpt', ns):
    caches.add_child(
        folium.CircleMarker(
            [float(cache.attrib['lat']), float(cache.attrib['lon'])],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6))
# add pop-up text to each marker on the map
#latitudes = list(df_incidents.Y)
#longitudes = list(df_incidents.X)
#labels = list(df_incidents.Category)

#for lat, lng, label in zip(latitudes, longitudes, labels):
#    folium.Marker([lat, lng], popup=label).add_to(sanfran_map)    
    
# add incidents to map
cacheMap.add_child(caches)